Inainte sa trimiti notebook-ul:

* asigura-te ca totul se executa cum trebuie. Prima oara, **restarteaza kernel-ul** (in meniu, selecteaza Kernel$\rightarrow$Restart & Run all);

* nu lasa niciun mesaj la output (`disp`, `printf`, linii care nu sunt incheiate cu `;`) in functiile pe care le scrii decat daca ti se specifica explicit sa faci acest lucru! Orice abatere va fi **depunctata automat** de grader;

* citeste instructiunile de [aici](https://github.com/radusqrt/numerical-methods/blob/master/lab_instructions.md) cu atentie;

* asigura-te ca rezolvi toate celulele in care scrie `YOUR CODE HERE` sau `YOUR ANSWER HERE` si ca iti treci numele mai jos:

In [1]:
STUDENT_NAME = "Babian Stelian-Alexandru";

---

# Laborator 4: Eliminari gaussiene
### Autor: Radu Stochitoiu

## Functii ajutatoare    

In [2]:
function [res] = isequal(A, B, err)
    res = norm(A - B) < err;
endfunction

function [x] = SST(A, b)
    n = length(b);
    x = [b(n) / A(n, n)];
    for i = n-1:-1:1
        x = [(b(i) - A(i, [i+1:n]) * x) / A(i,i); x];
    endfor
endfunction

## Problema 1

Implementati functia ```G(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana.

In [3]:
function [R, x] = G(A, b)
    [l,c]=size(A);
    R=zeros(l,l);
    extins=[A b];
    for i=1:l-1
        if extins(i,i)==0
            x=NaN;
            R=NaN;
        endif
        for p=i+1:l
            element=extins(p,i)/extins(i,i);
            extins(p,:)=extins(p,:)-element*extins(i,:);
        endfor
    endfor
    R=extins(:,1:l);
    b=extins(:,l+1);
    x=zeros(l,1);
    x=SST(R,b);  
endfunction

In [4]:
% Test 1
A = [2 1 7; 3 1 0; 5 1 4];
b = [3 2 4]';
R_ref = [2 1 7; 0 -0.5 -10.5; 0 0 18];
[R, x] = G(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 2

Implementati functia ```GPP(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana cu pivotare partiala.

In [5]:
function [R, x] = GPP(A, b)
   [l,c]=size(A);
   R=zeros(l,l);
   x=zeros(l,1);
   extins=[A,b];
   for i=1:l-1
       [piv,lpiv]=max(abs(extins(i:l,i)));
       lpiv=lpiv+i-1;
       aux=extins(i,:);
       extins(i,:)=extins(lpiv,:);
       extins(lpiv,:)=aux;
       if extins(i,i)==0
           x=NaN;
           R=NaN;
           return;
        endif
        for p=i+1:l
            element=extins(p,i)/extins(i,i);
            extins(p,:)=extins(p,:)-element*extins(i,:);
        endfor
    endfor
    R=extins(:,1:l);
    b=extins(:,l+1);
    x=SST(R,b);
endfunction

In [6]:
% Test 1
A = [0 1 3; 2 1 3; -3 1 1];
b = [3 1 1]';
R_ref = [-3 1 1; 0 5/3 11/3; 0 0 4/5];
[R, x] = GPP(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 3

Implementati functia ```GPPS(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana cu pivotare partiala scalata.

In [7]:
function [R, x] = GPPS(A, b)
    [l,c]=size(A);
    R=zeros(l,l);
    x=zeros(l,1);
    extins=[A,b];
    for i=1:l-1
        piv=-inf;
        lpiv=-1;
        for j=i:l
            if piv < abs(extins(j,i))/max(abs(extins(j,i:l)))
                piv=abs(extins(j,i))/max(abs(extins(j,i:l)));
                lpiv=j;
            endif
        endfor
        if(i~=lpiv)
            aux=extins(i,:);
            extins(i,:)=extins(lpiv,:);
            extins(lpiv,:)=aux;
        endif
        if extins(i,i)==0
            x=NaN;
            R=NaN;
            return;
        endif
        for p=i+1:l
            element=extins(p,i)/extins(i,i);
            extins(p,:)=extins(p,:)-element*extins(i,:);
        endfor
    endfor
    R=extins(:,1:l);
    b=extins(:,l+1);
    x=SST(R,b); 
endfunction

In [8]:
% Test 1
A = [3 -13 9 3; -6 4 1 -18; 6 -2 2 4; 12 -8 6 10];
b = [-19 -34 16 26]';
R_ref = [6 -2 2 4; 0 -12 8 1; 0 0 -2/3 5/3; 0 0 0 -3];
[R, x] = GPPS(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 4

Implementati functia ```GPT(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana cu pivotare totala (se pastreaza ordinea initiala a elementelor lui $x$).

In [9]:
function [R, x] = GPT(A, b)
    [l,c]=size(A);
    R=zeros(l,l);
    x=zeros(l,1);
    extins=[A,b];
    permutare=[1:l];
    for i=1:l-1
        [piv,lpiv]=max(abs(extins(i:l,i:l)));
        [piv,cpiv]=max(piv);
        lpiv=lpiv+i-1;
        lpiv=lpiv(cpiv);
        cpiv=cpiv+i-1;
        elem=extins(i,:);
        extins(i,:)=extins(lpiv,:);
        extins(lpiv,:)=elem;
        elem=extins(:,i);
        extins(:,i)=extins(:,cpiv);
        extins(:,cpiv)=elem;
        elem=permutare(i);
        permutare(i)=permutare(cpiv);
        permutare(cpiv)=elem;
        for p=i+1:l
            if abs(A(i,i))==0
                x=NaN;
                R=NaN;
                return;
            endif
            element=extins(p,i)/extins(i,i);
            extins(p,:)=extins(p,:)-element*extins(i,:);
        endfor
    endfor
    R=extins(:,1:l);
    b=extins(:,l+1);
    x=SST(R,b);
    lungime=length(x);
    y=zeros(l,1);
    for i=1:lungime
        y(permutare(i))=x(i);
    endfor
    x=y;   
endfunction

In [10]:
% Test 1
A = [2 1 7; 3 1 0; 5 1 4];
b = [3 2 4]';
R_ref = [7 2 1; 0 27/7 3/7; 0 0 2/3];
[R, x] = GPT(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 5 (BONUS)

Implementati functia ```GaussJordan(A)``` ce primeste ca parametru o matrice $A$ patratica, de lungime pozitiva, si returneaza inversa lui $A$ calculata prin procedeul Gauss-Jordan.

In [11]:
function [invA] = GaussJordan(A)
    % YOUR CODE HERE
    error('No Answer Given!')
endfunction

In [12]:
% Test 1
A = [2 3 0; 1 -2 -1; 2 0 -1];
[invA] = GaussJordan(A);
assert(isequal(invA, inv(A), 1e-5));

error: No Answer Given!
error: called from
    GaussJordan at line 3 column 5
error: 'invA' undefined near line 1 column 16
